In [1]:
!pwd

/home/niflheim/s183774/dft-superfluids/bandstructure


In [1]:
from gpaw import GPAW, PW, FermiDirac
from ase.io import read
from ase.spectrum.band_structure import BandStructurePlot
from ase.build import mx2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#choose
formula = 'MoS2'
vac = 20
out_dir = './out/'
name = out_dir + formula + '_PBE_gs.gpw'

In [3]:
#get structure
structure_params = {'MoS2': {'kind': '2H', 'a': 3.184, 'thickness': 3.127},
                       'MoSe2': {'kind': '2H', 'a': 3.320, 'thickness': 3.338},
                       'WS2': {'kind': '2H', 'a': 3.186, 'thickness': 3.359},
                       'WSe2': {'kind': '2H', 'a': 3.319, 'thickness': 3.146},
                       'BN': {'kind': '2H', 'a': 2.510, 'thickness': 1}}
s = structure_params[formula]
structure = mx2(formula=formula, kind=s['kind'], a=s['a'], thickness=s['thickness'],
            size=(1, 1, 1), vacuum=vac)

In [4]:
#choose path in kspace and load converged density
kpts = structure.cell.bandpath(path='MKG', npoints=30,
                           pbc=structure.pbc, eps=0.1)#what was eps??
calc = GPAW(name)

In [6]:
#calculate bandstructure
emptybands=26
convbands = emptybands // 2
parms = {
    'basis': 'dzp',
    'nbands': -emptybands,
    'txt': 'bs.txt',
    'fixdensity': True,
    'kpts': kpts,
    'convergence': {
        'bands': -convbands},
    'symmetry': 'off'}
calc = GPAW(name, **parms)
calc.get_potential_energy()

-21.96748480667283

In [9]:
Ef = calc.get_fermi_level()#fermi level
bs = calc.band_structure()

In [ ]:
bs_plt = BandStructurePlot(bs)
bs_plt.plot(emin=-6,emax=0)
bs_plt.finish_plot(show=True)

In [ ]:
bs_plt.finish_plot(filename=False,loc=False,show=True)

In [8]:
bs

BandStructure(path=BandPath(path='Kpt0Kpt1G', cell=[3x3], special_points={AGHKKpt0Kpt1LM}, kpts=[30x3]), energies=[1x30x39 values], reference=-3.2218974665439983)

In [9]:
Bands=bs.energies

In [10]:
np.shape(Bands)#(spin,k-points along path, number of bands=number of filled+number of empty)

(1, 30, 39)

In [48]:
#intermezzo
a=np.array([[1,2],[3,4]])
print(a[1,0])
print(a)
print(np.max(a,axis=0))

3
[[1 2]
 [3 4]]
[3 4]


In [11]:
Matrix_bandcol=np.array(Bands[0,:,:])
print(np.shape(Matrix_bandcol))
Vec_max_of_each_band=np.max(Matrix_bandcol,axis=0)
print(np.shape(Vec_max_of_each_band))
print(Vec_max_of_each_band)
#valence band is number 13!
n_val=np.sum(Vec_max_of_each_band<Ef)
print(n_val)

(30, 39)
(39,)
[-64.92997582 -39.10069287 -38.93654264 -38.93654247 -15.98777597
 -15.88676065  -8.91453835  -6.62219205  -6.50709107  -5.8070922
  -5.52011322  -4.96029434  -4.04657295  -1.29385101  -0.81958507
  -0.11690695   0.77202832   3.6713187    4.9942422    5.53862942
   6.35487358   7.07692177   7.56750953   7.60005506   7.76887424
   7.98744211   8.38806076   8.40883145   8.4089339    8.4091058
   8.49149515   8.68093158   8.73962381   8.90579974   9.02145175
   9.07408691   9.43302509   9.51824329   9.80053406]
13


In [12]:
band_valence=Matrix_bandcol[:,n_val-1]
band_conduction=Matrix_bandcol[:,n_val]

In [ ]:
#print(bs.path.kpts)
#plt.plot(range(len(band_valence)),band_valence)
#plt.plot(range(len(band_valence)),band_conduction)
#plt.show()

In [10]:
bs.path.kpts

array([[ 0.5       , -0.5       ,  0.        ],
       [ 0.48148148, -0.51851852,  0.        ],
       [ 0.46296296, -0.53703704,  0.        ],
       [ 0.44444444, -0.55555556,  0.        ],
       [ 0.42592593, -0.57407407,  0.        ],
       [ 0.40740741, -0.59259259,  0.        ],
       [ 0.38888889, -0.61111111,  0.        ],
       [ 0.37037037, -0.62962963,  0.        ],
       [ 0.35185185, -0.64814815,  0.        ],
       [ 0.33333333, -0.66666667,  0.        ],
       [ 0.31666667, -0.63333333,  0.        ],
       [ 0.3       , -0.6       ,  0.        ],
       [ 0.28333333, -0.56666667,  0.        ],
       [ 0.26666667, -0.53333333,  0.        ],
       [ 0.25      , -0.5       ,  0.        ],
       [ 0.23333333, -0.46666667,  0.        ],
       [ 0.21666667, -0.43333333,  0.        ],
       [ 0.2       , -0.4       ,  0.        ],
       [ 0.18333333, -0.36666667,  0.        ],
       [ 0.16666667, -0.33333333,  0.        ],
       [ 0.15      , -0.3       ,  0.   

In [ ]:
bs.plot(show=True)

In [8]:
#bs.write('MoS2_bs_early.json')